In [1]:
import random
import torch
import numpy as np 
import pandas as pd
from transformers import AutoModel
from time import perf_counter as timer
from sentence_transformers import util, SentenceTransformer

import textwrap
from is_tpt_ref import ReferenceClassifier

device = "cuda" if torch.cuda.is_available() else "cpu"

full_text_and_embeddings = pd.read_pickle("ajp_perc_prper_tpt_full_text_embeddings_2.pkl")
full_text_and_embeddings=full_text_and_embeddings.rename(columns={"Full Text Embedding":"embedding"})

# Load embeddings onto GPU
embeddings = torch.tensor(np.array(full_text_and_embeddings["embedding"].tolist()), dtype=torch.float32).to(device)

# Load model
#embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device=device)
# OR if you are using Jina:
embedding_model = AutoModel.from_pretrained("jinaai/jina-embeddings-v3", trust_remote_code=True, device_map=device)

df=full_text_and_embeddings

/home/f52r714/.conda/envs/lang_4/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native

In [2]:
csv=pd.read_excel("paper_labels_M_S_2.xlsx")

csv['Category (M)']=csv['Category (M)'].str.replace("*", "").replace("Journal business", "journal business").replace("Teacher", "teacher").replace('teaching', 'teacher').replace('Content', 'content').replace('content  ', 'content').replace('Student', 'student').replace("Content    ", "content").replace("Content  ", "content").replace("Teacher  ", "teacher").replace("Teacher ", "teacher").replace("Teaching", "teacher")


In [3]:
len(csv)

98

In [4]:
def check_doubles(csv):
    for doi in csv['Doi']:
        length=len(csv[csv['Doi']==doi]['Doi'])
        if length>1:
            print(doi)
#check_doubles(csv)

In [5]:

t1 = "Teaching students."
t2 = "Student focus."
t3 = "Physics content."
t4= "Journal business."

query_list = [t1, t2, t3, t4]

# Encode all query strings
s1="string standing wave frequency waves line tension aerator vibration transverse." #Teacher
s2="energy free representations ideas useful work different students conserved like." #Student
s3="force hypothetical natural object place velocity objects rock mass constant." #Physics content
#s4="Editorials, book reviews, announcements, obituaries. Journal business. Reports on business. "
s4="eye hyperopia high hyperopic lens course objects 43 clearly distance distant headaches lenses ." #Journal business


query_list_verbose=[s1, s2, s3, s4]
query_embedding = embedding_model.encode(query_list_verbose, convert_to_tensor=True)

print(f"Query: {query_list}")

# Compute dot product similarity
start_time = timer()
dot_scores_1 = util.dot_score(a=query_embedding, b=embeddings).T  # Shape: (n_embeddings, 3)
end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

# Add columns for s1, s2, s3, s4 similarity to dataframe
for n, t in enumerate(query_list):
    df[t]= dot_scores_1[:, n].cpu().numpy()


# Load classifier -- not currently used. This slows down batch processing time by a lot, so is recommended for smaller datasets. 
model_path = "trained_model.safetensors"
classifier = ReferenceClassifier(model_path)


def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)


Query: ['Teaching students.', 'Student focus.', 'Physics content.', 'Journal business.']
Time take to get scores on 43607 embeddings: 0.02978 seconds.


In [6]:
##But first we're going to calculate the topic scores for sentence chunk

# Default value of 'a'
a = -10

# Precompute the exponentials for efficiency
exp_values = np.exp(a * (1-df[query_list]))

# Compute the denominator for softmax-like normalization
denominator = exp_values.sum(axis=1)

# Create new score columns

for col in query_list:
    score_col = f"{col.strip()}_score"
    df[score_col] = exp_values[col] / denominator


In [7]:
len(df[df['doi']=='10.1119/1.2343490'])

#df ##Score calculated by taking eahc cosine for each sentence chunk

1

In [8]:
#Grabs labels form the CSV and inserts them into the dataframe. Renames the dataframe df_labels

weighted_score_ql=[]
for query in query_list:
    weighted_score_ql.append(f"{query}_score_weighted")


csv=csv.rename(columns={"Doi": "doi"})
csv['doi']=csv['doi'].str.replace(" ", "")
csv['doi']=csv['doi'].str.replace("\'", "")
csv['doi']=csv['doi'].str.replace("https://doi.org/", "")

df_labels=pd.merge(df, csv[['Category (M)', 'doi']], on='doi', how="left")

##At this point, the dataframe has handmade labels for the items that have received them by matching DOI. The labels are in single letter format
##the main topic labels need to be converted to this format 

In [9]:
weighted_score_ql

['Teaching students._score_weighted',
 'Student focus._score_weighted',
 'Physics content._score_weighted',
 'Journal business._score_weighted']

In [10]:
df_full_text=df_labels

In [11]:

def evaluate_labeling_recall(df, teacher_col, student_col, content_col, journal_col, main_group):
    # Mapping of category labels to full column names
    category_map = {
        't': teacher_col.strip(),
        's': student_col.strip(),
        'c': content_col.strip(),
        'jb': journal_col.strip()
    }

    def compare_labels(row):
        category = row['Category (M)']
        if pd.isna(category):
            return np.nan
        expected_group = category_map.get(category.strip().lower())
        return expected_group == row[main_group]

    df['Correctly labeled'] = df.apply(compare_labels, axis=1)
    valid = df[~df['Category (M)'].isna()].copy()

    print("Here:")
    print(len(valid))

    total = len(valid)
    correct = valid['Correctly labeled'].sum()
    percent_correct = 100 * correct / total if total > 0 else 0

    print(f"1. Total number of entries with a value in 'Category (M)': {total}")
    print(f"2. Percent correctly labeled (accuracy): {percent_correct:.2f}%")

    # Per-category accuracy (recall), precision, false positive rate
    print("3. Detailed metrics per category:")
    for cat, expected_val in category_map.items():
        # True Positives: predicted = expected = this category
        tp = valid[(valid['Category (M)'].str.lower() == cat) & (valid[main_group] == expected_val)]
        
        # False Negatives: actual is this category, but predicted is not
        fn = valid[(valid['Category (M)'].str.lower() == cat) & (valid[main_group] != expected_val)]

        # False Positives: predicted is this category, but actual is not
        fp = valid[(valid['Category (M)'].str.lower() != cat) & (valid[main_group] == expected_val)]

        # True Negatives: actual and predicted are both *not* this category
        tn = valid[(valid['Category (M)'].str.lower() != cat) & (valid[main_group] != expected_val)]

        tp_count = len(tp)
        fn_count = len(fn)
        fp_count = len(fp)
        tn_count = len(tn)

        recall = tp_count / (tp_count + fn_count) if (tp_count + fn_count) > 0 else 0
        precision = tp_count / (tp_count + fp_count) if (tp_count + fp_count) > 0 else 0
        fpr = fp_count / (fp_count + tn_count) if (fp_count + tn_count) > 0 else 0
        acc = (tp_count + tn_count) / (tp_count + tn_count + fp_count + fn_count)

        print(f"   {cat.title()}:")
        print(f"      Recall (TP / TP + FN): {recall:.2f}")
        print(f"      Precision (TP / TP + FP): {precision:.2f}")
        print(f"      False Positive Rate (FP / FP + TN): {fpr:.2f}")
        print(f"      Accuracy (TP + TN / All): {acc:.2f}")

    # Label distributions
    print("4. Label distribution:")
    cat_counts = valid['Category (M)'].str.lower().value_counts(normalize=True)
    main_counts = valid[main_group].value_counts(normalize=True)
    
    print("Category (M):")
    for k, v in cat_counts.items():
        print(f"     {k.title()}: {v:.2%}")
    print("Automated:")
    for k, v in main_counts.items():
        print(f"     {k[:40].strip()}: {v:.2%}")

    # Return doi and correctness
    result = valid[['doi', 'Correctly labeled']].reset_index(drop=True)
    return result

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import scipy.ndimage as nd
import numpy as np

  

def centroid_labeling_whole_text(df, K=1, whole_text=True):
    print(len(df))
    df_labels = df.copy()
    print(f"K: {K} whole_text: {whole_text}")

    # Step 1: Identify archetypal papers
    archetypal_papers = {}
    for label in query_list:
        k_actual = min(K, len(df_labels))
        top_k_dois = df_labels.sort_values(by=f"{label}_score", ascending=False).head(k_actual)['doi'].tolist()
        archetypal_papers[label] = top_k_dois
    #Hard coding in archetypal papers for Journal Business
    #archetypal_papers['Journal business.']=['10.1119/1.1987213', '10.1119/1.15470', '10.1119/1.1975265', '10.1119/1.2339284', '10.1119/1.18630', '10.1119/1.4972266']
    #archetypal_papers['Journal business.']=['10.1119/1.1987213', '10.1119/1.15470', '10.1119/1.1975265', '10.1119/1.2339284'] #'10.1119/1.18630', '10.1119/1.4972266']
    #copy pasting the papers that it chose through query 
        #archetypal_papers['Journal business.']=['10.1119/1.1934911', '10.1119/1.16066', '10.1119/1.1935913', '10.1119/1.1986075', '10.1119/1.2339284']
    
    print(archetypal_papers['Journal business.'])
    print("Here: ", archetypal_papers.keys())

    
    # Step 2: Compute centroids
    centroids = {}
    for label, dois in archetypal_papers.items():
        top_embeddings = []

        for doi in dois:
            if whole_text:
                # Use full-text embedding directly
                paper_embedding = df_labels[df_labels['doi'] == doi]['embedding'].values
                if len(paper_embedding) > 0:
                    top_embeddings.append(paper_embedding[0])
                    #print("Here")
                    #print(top_embeddings)
                else:
                    print("There's a problem")
            #print(top_embeddings)
            #print("hereere")
            centroid = np.mean(np.stack(top_embeddings), axis=0)
            centroids[label] = centroid

    # Step 3: Average embeddings per paper
    avg_embeddings = []

    # Step 4: Cosine similarity to centroids
    dot_prods={}
    for label in query_list:
        new_label=label+"_Advanced Centroid dotp"
        sent_emb=[item for item in df_labels['embedding']]
        #print(centroids[label])
        topic_emb=[centroids[label]]*len(sent_emb) #Copies the topic centroid embedding to be equal in length to sent_emb. For the case of the whole paper, this should just multiply it by one
        dot_prods[new_label]= np.diag(np.array(cosine_similarity(topic_emb, sent_emb)))  ##This uses cosine similarity instead of dot product. The vectors used here are normalized so that isn't a problem
    for new_label in [label+"_Advanced Centroid dotp" for label in query_list]:
        df_labels[new_label]=dot_prods[new_label]
    # Step 5: Softmax scoring
    #a = 10  # softmax scaling factor, can be adjusted if needed
    '''
    for item in sent_emb:  
        print(np.linalg.norm(item))
    for index in centroids.keys:
        for item in centroids[inded]:   
            print(np.linalg.norm(item))
    '''
    softmax_scores=[]
    ac_dotp_labels=[label+"_Advanced Centroid dotp" for label in query_list]
    for index, row in df_labels.iterrows():
        #print(row.keys())
        #print(type(row))
        #print(np.exp(a* (1-row[[label+"_Advanced Centroid dotp" for label in query_list]].values.astype(float))))
        #print("here")
        exp_values =np.exp(a* (1-row[ac_dotp_labels].values.astype(float)))
        #exp_values_dict={}
        #exp_values_dict[query_list+[" Advanced Centroid TS"]:exp_values]
        denominator = exp_values.sum(axis=0)
        #print(exp_values.sum(axis=0))
        #print(denominator)
        softmax_scores.append( exp_values / denominator )
    #print(softmax_scores)
    #df_labels[]=softmax_scores
    softmax_scores=np.flip(nd.rotate(np.array(softmax_scores),90), axis=0)
    print(softmax_scores[0])
    ac_ts_labels=[label+"_Advanced Centroid TS" for label in query_list]
    for n, label in enumerate(ac_ts_labels):
        df_labels[label]=softmax_scores[n]
    
    # Step 7: Identify main group
    df_labels['Main Group Advanced Centroid'] = df_labels[[f"{label}_Advanced Centroid TS" for label in query_list]].idxmax(axis=1)
    df_labels['Main Group Advanced Centroid'] = df_labels['Main Group Advanced Centroid'].str.replace("_Advanced Centroid TS", "")
    
    
    return df_labels


In [13]:
archetypal_papers=['10.1119/1.1934911', '10.1119/1.16066', '10.1119/1.1935913', '10.1119/1.1986075', '10.1119/1.2339284']

df_full_text_copy=df_full_text.copy()
print(len(df_full_text_copy))
df_full_text_copy=df_full_text_copy[(df_full_text_copy['doi'] != archetypal_papers[0]) & (df_full_text_copy['doi'] != archetypal_papers[1]) & (df_full_text_copy['doi'] != archetypal_papers[2]) & (df_full_text_copy['doi'] != archetypal_papers[3]) & (df_full_text_copy['doi'] != archetypal_papers[4])]
print(len(df_full_text_copy))

43607
43602


In [14]:

test=centroid_labeling_whole_text(df_full_text_copy, K=4)

43602
K: 4 whole_text: True
['10.1119/1.1855743', '10.1119/1.4795378', '10.1119/1.15195', '10.1119/1.1484145']
Here:  dict_keys(['Teaching students.', 'Student focus.', 'Physics content.', 'Journal business.'])
[0.27158646 0.26518625 0.09618734 ... 0.0618028  0.04097862 0.05596693]


In [15]:
df_full=test.merge(df_labels[['doi', 'Category (M)']], how='left', on="doi")

mg1="Main Group Advanced Centroid" #Advanced centroids based on full text
mg2="MG Score Full Embedding" #Non advanced centroid, labels from full text
mg3="MG Score Avg Cos" #Centroid based entirely on queries, classification based on scoring each sentence chunk and averaging those scores

df1=df_full



In [16]:
df_full.keys()
df_full=df_full.rename(columns={"Category (M)_x": "Category (M)"})

In [17]:
mg1="Main Group Advanced Centroid" #Advanced centroids based on full text
mg2="MG Score Full Embedding" #Non advanced centroid, labels from full text
mg3="MG Score Avg Cos" #Not advanced, Chunked


df1=df_full
#mg="MG Score Full Embedding"

##This is Group D, the one that matters
results = evaluate_labeling_recall(
    df1,
    query_list[0], #teacher_col
    query_list[1], #student_col
    query_list[2], #content_col
    query_list[3], #journal_col
    mg1)


Here:
98
1. Total number of entries with a value in 'Category (M)': 98
2. Percent correctly labeled (accuracy): 52.04%
3. Detailed metrics per category:
   T:
      Recall (TP / TP + FN): 0.30
      Precision (TP / TP + FP): 0.41
      False Positive Rate (FP / FP + TN): 0.13
      Accuracy (TP + TN / All): 0.73
   S:
      Recall (TP / TP + FN): 1.00
      Precision (TP / TP + FP): 0.55
      False Positive Rate (FP / FP + TN): 0.16
      Accuracy (TP + TN / All): 0.87
   C:
      Recall (TP / TP + FN): 0.64
      Precision (TP / TP + FP): 0.53
      False Positive Rate (FP / FP + TN): 0.29
      Accuracy (TP + TN / All): 0.68
   Jb:
      Recall (TP / TP + FN): 0.27
      Precision (TP / TP + FP): 0.58
      False Positive Rate (FP / FP + TN): 0.07
      Accuracy (TP + TN / All): 0.76
4. Label distribution:
Category (M):
     C: 33.67%
     Jb: 26.53%
     T: 23.47%
     S: 16.33%
Automated:
     Physics content.: 40.82%
     Student focus.: 29.59%
     Teaching students.: 17.35%
   

In [18]:
csv=csv.rename(columns={"Doi": "doi"})

In [19]:
#Checking to see where the missing entries of Category (M) are going

import pandas as pd
import numpy as np

def find_missing_M_val(df1, df2):
    def find_missing_elements(list1, list2):
        set1 = set(list1)
        set2 = set(list2)
        
        missing_in_list1 = list(set2 - set1)
        missing_in_list2 = list(set1 - set2)
    
        return missing_in_list1, missing_in_list2
    df1[pd.notna(df1['Category (M)']) ==True]['doi']
    df1_values=df1[pd.notna(df1['Category (M)']) ==True]['doi']
    df2_values=df2[pd.notna(df2['Category (M)']) ==True]['doi']

    
    
    return find_missing_elements(df1_values, df2_values)
find_missing_M_val(df1, csv)

([], [])

In [20]:
full_text_and_embeddings[full_text_and_embeddings['doi']=='10.1119/1.2343490']

,full_text,doi,year,journal,char_count,token_count,sentences,sentences_count,sentence_chunks,num_chunks,embedding,Teaching students.,Student focus.,Physics content.,Journal business.,Teaching students._score,Student focus._score,Physics content._score,Journal business._score
12999,Nan,10.1119/1.2343490,NaN,tpt,3,0.75,[Nan],1,[[Nan]],1,"[tensor(0.0063), tensor(-0.1650), tensor(0.025...",0.364036,0.399712,0.329448,0.393104,0.223531,0.319359,0.158171,0.298939


import joblib

joblib.dump(df1, "full_text_refined_df.pkl")

In [21]:
df1

,full_text,doi,year,journal,char_count,token_count,sentences,sentences_count,sentence_chunks,num_chunks,...,Student focus._Advanced Centroid dotp,Physics content._Advanced Centroid dotp,Journal business._Advanced Centroid dotp,Teaching students._Advanced Centroid TS,Student focus._Advanced Centroid TS,Physics content._Advanced Centroid TS,Journal business._Advanced Centroid TS,Main Group Advanced Centroid,Category (M)_y,Correctly labeled
0,Make a Mystery Circuit with a Bar Light Fixtur...,10.1119/1.2715425,NaN,tpt,7234,1808.50,[Make a Mystery Circuit with a Bar Light Fixtu...,81,[[Make a Mystery Circuit with a Bar Light Fixt...,17,...,0.567285,0.453943,0.485728,0.271586,0.412857,0.132913,0.182644,Student focus.,NaN,NaN
1,AGOLDEN OLDIE-ABLAOK BOX OIROUIT \r\nClifton K...,10.1119/1.2343976,NaN,tpt,4633,1158.25,[AGOLDEN OLDIE-ABLAOK BOX OIROUIT \r\nClifton ...,58,[[AGOLDEN OLDIE-ABLAOK BOX OIROUIT \r\nClifton...,12,...,0.539128,0.462512,0.479265,0.265186,0.364788,0.169552,0.200474,Student focus.,NaN,NaN
2,Modeling Electricity: Model-Based Inquiry with...,10.1119/1.4745686,NaN,tpt,17496,4374.00,[Modeling Electricity: Model-Based Inquiry wit...,140,[[Modeling Electricity: Model-Based Inquiry wi...,28,...,0.681222,0.480798,0.416193,0.096187,0.749807,0.101046,0.052959,Student focus.,NaN,NaN
3,"Two Approaches to Learning Physics \r\n""I look...",10.1119/1.2342910,NaN,tpt,37322,9330.50,"[Two Approaches to Learning Physics \r\n""I loo...",324,"[[Two Approaches to Learning Physics \r\n""I lo...",65,...,0.702052,0.697663,0.569715,0.053867,0.425555,0.407280,0.113298,Student focus.,NaN,NaN
4,"\r\nJochen Kuhn and Patrik Vogt, Column Editor...",10.1119/1.4865529,NaN,tpt,6621,1655.25,"[\r\nJochen Kuhn and Patrik Vogt, Column Edito...",76,"[[\r\nJochen Kuhn and Patrik Vogt, Column Edit...",16,...,0.461446,0.533366,0.434795,0.385857,0.160819,0.330129,0.123195,Teaching students.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43597,Examining faculty choices while implementing t...,10.1119/perc.2023.pr.Willison,2023.0,perc,24881,6220.25,[Examining faculty choices while implementing ...,203,[[Examining faculty choices while implementing...,41,...,0.579572,0.423246,0.323882,0.057473,0.732347,0.153391,0.056789,Student focus.,NaN,NaN
43598,Analyzing the dimensionality of the Energy and...,10.1119/perc.2023.pr.Wu,2023.0,perc,24460,6115.00,[Analyzing the dimensionality of the Energy an...,227,[[Analyzing the dimensionality of the Energy a...,46,...,0.687440,0.465194,0.407578,0.023987,0.834744,0.090438,0.050831,Student focus.,NaN,NaN
43599,Students’ use of symmetry as a tool for sensem...,10.1119/perc.2023.pr.Young,2023.0,perc,27598,6899.50,[Students’ use of symmetry as a tool for sense...,264,[[Students’ use of symmetry as a tool for sens...,53,...,0.626704,0.428842,0.421718,0.061803,0.740479,0.102379,0.095339,Student focus.,NaN,NaN
43600,Analyzing Physics Majors’ Specialization Low I...,10.1119/perc.2023.pr.Zohrabi_Alaee,2023.0,perc,27954,6988.50,[Analyzing Physics Majors’ Specialization Low ...,222,[[Analyzing Physics Majors’ Specialization Low...,45,...,0.597576,0.474457,0.412088,0.040979,0.662116,0.193302,0.103603,Student focus.,NaN,NaN
